## Project Overview



## Import Libraries

In [1]:
import pandas as pd
import geopandas as gpd
from pathlib import Path
import os
from __future__ import annotations

# Set the working directory to the project root
project_root = Path().resolve().parent
os.chdir(project_root)

# import local modules
import prospectivity_tools as pt

In [2]:
# Read in geopackage
gdf = gpd.read_file(pt.settings.paths["input_gpkg"], engine="pyogrio").to_crs(pt.settings.crs)

In [3]:
##PARAMETERS (todo)

# adjustable fall-off distance parameter. 
# Fuzzy Gamma? https://link.springer.com/article/10.1007/s11053-023-10255-8#:~:text=match%20at%20L755%20geophysical%20response,intermediate%20layers%20were%20then%20combined

In [4]:
## Todo clean up
## Turn into a function
## Feature engineering

# ------------------------------------------------------------------
# Lithology flags (ultramafic & granodiorite proximity drivers)
# ------------------------------------------------------------------
# Build a lowercase text blob for keyword searches
text_cols = ["rock_type", "unit_desc", "strat_name"]
gdf["_search"] = (
    gdf[text_cols]
    .fillna("")
    .agg(" | ".join, axis=1)
    .str.lower()
)

# Binary indicators for target lithologies
gdf["is_ultramafic"] = gdf["_search"].str.contains(
    r"(?:ultramafic|serpentinite|peridotite)"
).astype(int)

gdf["is_granodiorite"] = gdf["_search"].str.contains(
    r"\bgranodiorite\b"
).astype(int)

# ------------------------------------------------------------------
# Rock‑class back‑fill (simple heuristic)
# ------------------------------------------------------------------
class_map = {"granodiorite": "plutonic", "serpentinite": "metamorphic"}


# todo is this right??
def infer_class(row):
    if pd.isna(row["rock_class"]):
        for k, v in class_map.items():
            if k in row["_search"]:
                return v
    return row["rock_class"]

gdf["inferred_rock_class"] = gdf.apply(infer_class, axis=1)
gdf.drop(columns=["_search"], inplace=True)

# Keep track of generated columns
generated_cols = [
    "is_ultramafic",
    "is_granodiorite",
    "inferred_rock_class",
]

In [5]:
gen_columns_description = pt.df_more_info(gdf[generated_cols])
print(gen_columns_description)

Column: is_ultramafic
  Min: 0, Max: 1, Mean: 0.13972431077694236


Column: is_granodiorite
  Min: 0, Max: 1, Mean: 0.15852130325814537


Column: inferred_rock_class
  Value counts:
sedimentary rocks    556
intrusive rocks      455
volcanic rocks       435
metamorphic rocks    104
ultramafic rocks      46
Name: inferred_rock_class, dtype: int64





In [6]:
# Load rock polygons
# todo: rename function
rock_a, rock_b = pt.load_bedrock(gdf)

In [7]:
# Build the hexagon grid covering both rock types
grid = pt.build_grid(rock_a, rock_b)

In [8]:
# Compute distances to each rock type
grid = pt.add_distance_columns(grid)

In [9]:
# Calculate likelihood scores using the Gaussian kernel
scored = pt.compute_likelihood(grid)

In [10]:
# Persist results
out_path = pt.write_parquet(scored)

# Create an interactive map
html_path = pt.build_map(scored)

Generated 6041 features for visualization


In [11]:
html_path